In [1]:
from numpy import argmax
from pickle import load
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from tensorflow.keras.backend import sum
from nltk.translate.bleu_score import corpus_bleu

In [2]:
# Load doc into memory
def load_doc(filename):

    file = open(filename, 'r')
    text = file.read()
    file.close()
    
    return text

# Load the list of photo identifiers
def load_set(filename):
    
    doc = load_doc(filename)
    dataset = list()

    for line in doc.split('\n'):
        if len(line) < 1:
            continue

        identifier = line.split('.')[0]
        dataset.append(identifier)
        
    return set(dataset)

# Load clean descriptions
def load_clean_descriptions(filename, dataset):

    doc = load_doc(filename)
    descriptions = dict()
    
    for line in doc.split('\n'):

        tokens = line.split()
        image_id, image_desc = tokens[0], tokens[1:]

        if image_id in dataset:
            if image_id not in descriptions:
                descriptions[image_id] = list()
                
            # wrap description in tokens
            desc = 'startseq ' + ' '.join(image_desc) + ' endseq'

            descriptions[image_id].append(desc)
            
    return descriptions

# Load photo features
def load_photo_features(filename, dataset):

    all_features = load(open(filename, 'rb'))
    features = {k: all_features[k] for k in dataset}
    
    return features

# Creating a list of descriptions
def to_lines(descriptions):
    
    all_desc = list()
    
    for key in descriptions.keys():
        [all_desc.append(d) for d in descriptions[key]]
        
    return all_desc

# Fit a tokenizer given caption descriptions
def create_tokenizer(descriptions):
    
    lines = to_lines(descriptions)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    
    return tokenizer

# Length of the description with the most words
def max_length(descriptions):
    
    lines = to_lines(descriptions)
    
    return max(len(d.split()) for d in lines)

# Map an integer to a word
def word_for_id(integer, tokenizer):
    
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
        
    return None

# Generate image description
def generate_desc(model, tokenizer, photo, max_length):

    in_text = 'startseq'

    for i in range(max_length):

        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)

        yhat = model.predict([photo,sequence], verbose=0)
        yhat = argmax(yhat)
        word = word_for_id(yhat, tokenizer)

        if word is None:
            break
            
        in_text += ' ' + word
        
        if word == 'endseq':
            break
            
    return in_text

In [3]:
# Evaluating the model
def evaluate_model(model, descriptions, photos, tokenizer, max_length):
    
    actual, predicted = list(), list()
    
    for key, desc_list in descriptions.items():
        yhat = generate_desc(model, tokenizer, photos[key], max_length)
        
        # Actual and predicted values
        references = [d.split() for d in desc_list]
        actual.append(references)
        predicted.append(yhat.split())
        
    # Calculating the BLEU score
    print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
    print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
    print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
    print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

In [4]:
# Preparing tokenizer on train set
filename = 'Flickr8k_text/Flickr_8k.trainImages.txt'
train = load_set(filename)
print('Dataset: %d' % len(train))

# Descriptions
train_descriptions = load_clean_descriptions('descriptions.txt', train)
print('Descriptions: train=%d' % len(train_descriptions))

# Prepare tokenizer
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

# Determine the maximum sequence length
max_length = max_length(train_descriptions)
print('Description Length: %d' % max_length)

# Prepare test set
# Test set
filename = 'Flickr8k_text/Flickr_8k.testImages.txt'
test = load_set(filename)
print('Dataset: %d' % len(test))

# Descriptions
test_descriptions = load_clean_descriptions('descriptions.txt', test)
print('Descriptions: test=%d' % len(test_descriptions))

# Photo features
test_features = load_photo_features('features.pkl', test)
print('Photos: test=%d' % len(test_features))

# Load the model
filename = 'model_19.h5'
model = load_model(filename)

# Evaluate model
evaluate_model(model, test_descriptions, test_features, tokenizer, max_length)

Dataset: 6000
Descriptions: train=6000
Vocabulary Size: 7579
Description Length: 34
Dataset: 1000
Descriptions: test=1000
Photos: test=1000
BLEU-1: 0.561116
BLEU-2: 0.304228
BLEU-3: 0.206748
BLEU-4: 0.093943
